In [1]:
from IPython.display import display, clear_output
import numpy as np
import pyopencl as cl
from PIL import Image
import time
import signal

In [2]:
from skvideo.io import FFmpegWriter
from IPython.display import HTML
import base64
import io

In [3]:
signal_done = False

def signal_handler(signal, frame):
    global signal_done
    signal_done = True

def stop_on_signal():
    global signal_done
    signal_done = False
    signal.signal(signal.SIGINT, signal_handler)

In [4]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags

In [5]:
class Buffer:
    def __init__(self, nparray, ro=False, dual=False):
        self.ro = ro
        self.dual = dual
        self.host = nparray
        flags = 0
        if ro:
            flags |= mf.READ_ONLY
        else:
            flags |= mf.READ_WRITE
        mkbuf = lambda buf: cl.Buffer(ctx, flags | mf.COPY_HOST_PTR, hostbuf=buf)
        self.buf = mkbuf(self.host)
        if dual:
            self.dbuf = mkbuf(self.host)
    
    def swap(self):
        if self.dual:
            self.buf, self.dbuf = self.dbuf, self.buf
    
    def load(self):
        cl.enqueue_copy(queue, self.host, self.buf)

In [61]:
class Scene:
    def __init__(self, size, params):
        self.size = size
        self.shape = (size[1], size[0])
        
        self.params = params
        
        with open("simulation.cl", "r") as f:
            self.program = cl.Program(ctx, f.read()).build()
        
        self.buffers = {}
        
        self.buffers["color"] = Buffer(np.zeros((*self.shape, 4), dtype=np.float32), dual=True)
        self.buffers["velocity"] = Buffer(np.zeros((*self.shape, 2), dtype=np.float32), dual=True)
        self.buffers["pressure"] = Buffer(np.zeros((*self.shape, 2), dtype=np.float32), dual=True)
        self.buffers["screen"] = Buffer(np.zeros((*self.shape, 3), dtype=np.uint8))
        
        self.buffers["C"] = Buffer(np.array([
            self.params["time_step"],
            self.params["grid_size"],
            self.params["viscosity"],
        ], dtype=np.float32))
        
        self.program.put_circle(
            queue,
            self.size,
            None,
            
            self.buffers["C"].buf,
            self.buffers["color"].buf,
            self.buffers["velocity"].buf,
            
            np.array([size[0]/2, size[1]/2], dtype=np.float32),
            np.array([100], dtype=np.float32),
            np.array([0,0,1,0], dtype=np.float32),
            np.array([1.0,1.0], dtype=np.float32),
        )
        
    def step(self):
        self.program.diffuse(
            queue,
            self.size,
            None,
            
            self.buffers["C"].buf,
            self.buffers["velocity"].buf,
            self.buffers["velocity"].dbuf,
        )
        self.buffers["velocity"].swap()
        
    def draw(self):
        self.program.render(
            queue,
            self.size,
            None,
            
            self.buffers["C"].buf,
            self.buffers["color"].buf,
            self.buffers["velocity"].buf,
            self.buffers["screen"].buf,
        )
        self.buffers["screen"].load()
        return self.buffers["screen"].host

In [71]:
scene = Scene((800, 600), params={
    "time_step": 1e-3,
    "grid_size": 0.001,
    "viscosity": 1e-4,
})

In [72]:
params = {
    "-vcodec": "libx264",
    "-pix_fmt": "yuv420p",
    "-profile:v": "baseline",
    "-level": "3"
}
video = FFmpegWriter("tmp.mp4", outputdict=params)
stride = int(1.0/(24*scene.params["time_step"]))
for i in range(4*24):
    for j in range(stride):
        scene.step()
    img = scene.draw()
    video.writeFrame(img)
video.close()

In [ ]:
with open("tmp.mp4", "rb") as f:
    vdata = f.read()
vbase64 = base64.b64encode(vdata).decode("ascii")
HTML('<video controls src="data:video/mp4;base64,%s" type="video/mp4" >' % vbase64)

In [ ]:
display(Image.fromarray(scene.draw()))